<a href="https://colab.research.google.com/github/iserveradmin/GDG/blob/main/ahmedabad_gdg_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q --upgrade sentence-transformers faiss-cpu
%pip install -q "numpy==2.0.2" "pandas==2.2.2" "scikit-learn>=1.6"

In [ ]:
import pandas as pd
import re

# Seed Q&A based on real doctors & hospitals in Ahmedabad
qa_data = [
    {
        "question": "Best cardiologist near SG Highway Ahmedabad for heart checkup?",
        "Answer": "You can visit the cardiology team at KD Hospital on S. G. Highway, Ahmedabad, where specialists like Dr. Anand Shukla and Dr. Gaurav Panchal consult for heart checkups."
    },
    {
        "question": "Need a heart specialist in Ahmedabad for my father with chest pain",
        "Answer": "For chest pain and heart issues, you can consult senior cardiologists in Ahmedabad such as Dr. Jayesh Prajapati or Dr. Sameer Dani, who practice at reputed cardiac centres in the city."
    },
    {
        "question": "Cardiologist in Ahmedabad for follow-up after angioplasty",
        "Answer": "For post-angioplasty follow-up, you can consult experienced cardiologists like Dr. Tejas Patel or doctors at CIMS Hospital and other major heart hospitals in Ahmedabad."
    },
    {
        "question": "Best hospital for heart emergency near Satellite, Ahmedabad",
        "Answer": "Shalby Hospital on Satellite Road, Ahmedabad is a well-known multi-speciality hospital with emergency and cardiology services for heart-related emergencies."
    },
    {
        "question": "Good child specialist in Satellite Ahmedabad for vaccination tomorrow",
        "Answer": "You can book an appointment with pediatricians like Dr. Harshil Savalia or Dr. Shirish Parikh in Satellite, Ahmedabad, both known for child vaccination and routine checkups."
    },
    {
        "question": "Looking for a pediatrician near Bopal Ahmedabad for fever in child",
        "Answer": "You can consult a pediatrician at Sannidhya Maternity & Multi Speciality Hospital in Bopal or nearby Satellite, where doctors like Dr. Harshil Savalia and Dr. Ashish Mahuvakar see children."
    },
    {
        "question": "Need experienced pediatrician in Ahmedabad for newborn checkup",
        "Answer": "For newborn care, you can visit Dr. Nishchal Bhatt Children Hospital in Satellite or other senior pediatricians in Satellite and Navrangpura areas of Ahmedabad."
    },
    {
        "question": "Best gynecologist on SG Highway Ahmedabad for pregnancy checkup",
        "Answer": "You can consult gynecologists on S. G. Highway such as Dr. Dipti Jain or Dr. Anita Shukla, who practice in reputed maternity and women’s hospitals along SG Highway in Ahmedabad."
    },
    {
        "question": "Gynecologist in Ahmedabad for high-risk pregnancy near SG Highway",
        "Answer": "For high-risk pregnancy, you can visit SG Shalby Hospital on S. G. Highway, Ahmedabad and consult specialists like Dr. Juhi Patel or other senior obstetricians there."
    },
    {
        "question": "Where can I find a good women’s hospital in Ahmedabad for IVF and pregnancy?",
        "Answer": "You can check Divine Women’s Hospital in Ahmedabad, which offers IVF, maternity and high-risk pregnancy care, along with other dedicated women’s hospitals in the city."
    },
    {
        "question": "Need dermatologist in Navrangpura Ahmedabad for acne treatment",
        "Answer": "You can consult dermatologists like Dr. Anshul Warman or Dr. Hiren N. Patel in Navrangpura, Ahmedabad, who are known for skin, acne and cosmetic dermatology treatments."
    },
    {
        "question": "Best skin specialist in Ahmedabad near Navrangpura for hair fall",
        "Answer": "In Navrangpura, you can visit clinics such as Terra Skin and Aesthetics Clinic or Dr. Shikha’s Skin Clinic, which have dermatologists experienced in hair fall and scalp issues."
    },
    {
        "question": "Looking for dermatologist in Ahmedabad for laser treatment",
        "Answer": "You can search for dermatology clinics in Navrangpura and Satellite such as Silk Skin Laser Cosmetology Clinic or other laser-focused skin centres in Ahmedabad."
    },
    {
        "question": "ENT specialist in Ahmedabad for ear pain near Maninagar",
        "Answer": "For ear pain, you can consult an ENT specialist at multi-speciality hospitals or ENT clinics in Maninagar and nearby areas of Ahmedabad."
    },
    {
        "question": "Where can I find a neurologist in Ahmedabad for migraine problems?",
        "Answer": "You can visit large multi-speciality hospitals or neurology clinics in areas like SG Highway, Navrangpura or Satellite, where neurologists treat migraine and nerve-related issues."
    },
    {
        "question": "Need orthopedic doctor in Ahmedabad for knee pain near Thaltej",
        "Answer": "You can consult an orthopedic specialist at multi-speciality hospitals in Thaltej and nearby SG Highway, Ahmedabad for knee pain evaluation and treatment."
    },
    {
        "question": "Which hospital in Ahmedabad is good for general physician consultation near Navrangpura?",
        "Answer": "Navrangpura and surrounding areas have several multi-speciality hospitals and clinics with general physicians for fever, cold and routine health checkups."
    },
    {
        "question": "Best eye doctor in Ahmedabad for vision check near Prahlad Nagar",
        "Answer": "You can search for ophthalmology clinics and eye hospitals around Prahlad Nagar and SG Highway in Ahmedabad for routine eye checkups and vision correction."
    },
    {
        "question": "Where can I find a psychiatrist in Ahmedabad for anxiety issues?",
        "Answer": "You can consult psychiatrists in areas like Navrangpura, Satellite, and SG Highway, where mental health clinics and multi-speciality hospitals provide counseling and medication for anxiety."
    },
    {
        "question": "Doctor in Ahmedabad for diabetes management near Maninagar",
        "Answer": "For diabetes, you can consult an endocrinologist or diabetologist in Maninagar or nearby areas, or visit large multi-speciality hospitals in central Ahmedabad."
    },
    {
        "question": "Good maternity doctor in Ahmedabad for normal delivery near Satellite",
        "Answer": "You can visit maternity and women’s hospitals in Satellite, Ahmedabad such as Shalby Hospital or dedicated women’s clinics, where gynecologists manage pregnancy and normal deliveries."
    },
    {
        "question": "Looking for pediatric emergency services in Ahmedabad at night",
        "Answer": "For pediatric emergencies at night, you should visit large multi-speciality hospitals or children’s hospitals in areas like Satellite, Navrangpura or SG Highway that offer 24x7 emergency care."
    },
    {
        "question": "Where can I get skin allergy treatment in Ahmedabad near Navrangpura?",
        "Answer": "Dermatologists in Navrangpura, like those at Terra Skin and Aesthetics Clinic or similar skin clinics, treat skin allergies and rashes."
    },
    {
        "question": "Need a gynecologist for PCOS treatment in Ahmedabad near SG Highway",
        "Answer": "You can consult gynecologists along SG Highway, Ahmedabad such as those practicing at SG Shalby Hospital or other women’s clinics there for PCOS and hormonal issues."
    },
    {
        "question": "Best hospital in Ahmedabad for comprehensive cardiac checkup package",
        "Answer": "You can explore cardiac checkup packages at major heart hospitals in Ahmedabad such as CIMS Hospital, Apex Heart Institute, Jivraj Mehta Hospital or similar cardiac centres."
    },
]

# Define Ahmedabad-focused localities and generic specialties for programmatic expansion
ahmedabad_areas = [
    "Satellite, Ahmedabad",
    "S G Highway, Ahmedabad",
    "Navrangpura, Ahmedabad",
    "Bopal, Ahmedabad",
    "Prahlad Nagar, Ahmedabad",
    "Maninagar, Ahmedabad",
    "Thaltej, Ahmedabad",
    "Gota, Ahmedabad",
    "Chandkheda, Ahmedabad",
    "Nikol, Ahmedabad",
]

specialties = [
    ("diabetes or sugar problem", "Endocrinologist or Diabetologist"),
    ("heart issues or chest pain", "Cardiologist"),
    ("stomach pain or acidity", "Gastroenterologist"),
    ("skin problems or acne", "Dermatologist"),
    ("children health or vaccination", "Pediatrician"),
    ("bones and joint pain", "Orthopedic specialist"),
    ("ear pain or throat infection", "ENT specialist"),
    ("mental health, stress or anxiety", "Psychiatrist"),
    ("eyes checkup or blurred vision", "Ophthalmologist"),
    ("fever, cold and cough", "General Physician"),
]
for area in ahmedabad_areas:
    for condition, doctor in specialties:
        qa_data.append({
            "question": f"I need a doctor for {condition} in {area}",
            "Answer": f"You should consult a {doctor} in {area} for {condition} treatment."
        })

# Ensure we have at least ~100 entries
qa_data = qa_data[:100]

# Create DataFrame
df = pd.DataFrame(qa_data)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Apply preprocessing to questions
df['question'] = df['question'].apply(preprocess_text)

display(df.head())

,question,Answer
0,best cardiologist near sg highway ahmedabad fo...,You can visit the cardiology team at KD Hospit...
1,need a heart specialist in ahmedabad for my fa...,"For chest pain and heart issues, you can consu..."
2,cardiologist in ahmedabad for followup after a...,"For post-angioplasty follow-up, you can consul..."
3,best hospital for heart emergency near satelli...,"Shalby Hospital on Satellite Road, Ahmedabad i..."
4,good child specialist in satellite ahmedabad f...,You can book an appointment with pediatricians...


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re

# Load a pre-trained sentence transformer model
# You can try different models here, e.g., 'all-mpnet-base-v2' for potentially better results
model = SentenceTransformer('all-MiniLM-L6-v2')

# Preprocessing function (duplicate for clarity, ideally in a shared utility)
def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Generate embeddings for the preprocessed questions
question_embeddings = model.encode(df['question'].tolist())

# Get the dimensionality of the embeddings
dimension = question_embeddings.shape[1]

# Create a FAISS index (using a simple IndexFlatL2 for demonstration)
index = faiss.IndexFlatL2(dimension)

# Add the embeddings to the index
index.add(question_embeddings)

print(f"Number of embeddings in the index: {index.ntotal}")

Number of embeddings in the index: 100


In [ ]:
# Define a query question
query_question = "i have cold. suggest me a doctor"

# Generate embedding for the query question
query_embedding = model.encode([preprocess_text(query_question)]) # Preprocess the query

# Perform a similarity search (e.g., find the 3 most similar questions)
k = 3  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

# Retrieve the corresponding answers from the original DataFrame
print(f"Query: {query_question}")
print("\nBest matching questions and their answers:")
for i in range(k):
    print(f"Answer: {df['Answer'].iloc[indices[0][i]]}")
    print(f"Distance: {distances[0][i]:.4f}")
    print("-" * 20)

Query: i have cold. suggest me a doctor

Best matching questions and their answers:
Answer: You should consult a General Physician in Prahlad Nagar, Ahmedabad for fever, cold and cough treatment.
Distance: 0.9181
--------------------
Answer: You should consult a General Physician in Thaltej, Ahmedabad for fever, cold and cough treatment.
Distance: 0.9466
--------------------
Answer: You should consult a General Physician in Satellite, Ahmedabad for fever, cold and cough treatment.
Distance: 0.9782
--------------------


In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

# Ensure the GOOGLE_API_KEY is set up in Colab secrets
try:
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"Error setting up Gemini API key: {e}")
    print("Please ensure you have set up the GOOGLE_API_KEY correctly in Colab secrets.")
    # Exit the cell execution if API key setup fails
    raise

# Initialize the Gemini API
# You can choose a different model if needed, e.g., 'gemini-pro'
try:
    gemini_model = genai.GenerativeModel('gemini-2.5-flash')
except Exception as e:
    print(f"Error initializing Gemini model: {e}")
    # Exit the cell execution if model initialization fails
    raise


# Get the original query and the top answers from the previous search
# Assuming 'query_question', 'indices', and 'k' are available from the previous search cell (3a6d003c)
query = query_question
retrieved_answers = [df['Answer'].iloc[indices[0][i]] for i in range(k)]

# Craft a prompt for the LLM, emphasizing the use of only the provided context
prompt = f"""
Based ONLY on the following query and retrieved information, provide a helpful and conversational response.
Do NOT include any information not present in the provided context.

Query: "{query}"

Retrieved Information:
{'- ' + '\\n- '.join(retrieved_answers)}

Please synthesize this information and provide a clear answer to the user's query, strictly using the provided context.
"""

# Generate a response using the LLM
try:
    response = gemini_model.generate_content(prompt)
    print("LLM's Response (based only on provided context):")
    print(response.text)
except Exception as e:
    print(f"An error occurred while generating the LLM response: {e}")
    print("Please ensure you have set up the GOOGLE_API_KEY correctly in Colab secrets and have run the cells to initialize the model.")

LLM's Response (based only on provided context):
I'm sorry to hear you have a cold. Based on the information I have, you should consult a General Physician. You can find one in Prahlad Nagar, Thaltej, or Satellite, all in Ahmedabad, for fever, cold, and cough treatment.


In [ ]:
import gradio as gr

# take input from user as question
#initialized the prompt and take as input
#submit query to gemeni
#deliver response

def search_ui(query_ui):
  # Generate embedding for the query question
  query_embedding_ui = model.encode([preprocess_text(query_ui)]) # Preprocess the query

# Perform a similarity search (e.g., find the 3 most similar questions)
  k = 3  # Number of nearest neighbors to retrieve
  distances, indices = index.search(query_embedding_ui, k)

# Retrieve the corresponding answers from the original DataFrame
  print(f"Query: {query_ui}")
  print("\nBest matching questions and their answers:")
  retrieved_answers = [df['Answer'].iloc[indices[0][i]] for i in range(k)]
  prompt_ui = f"""
  Based ONLY on the following query and retrieved information, provide a helpful and conversational response.Do NOT include any information not present in the provided context.
  Query: "{query_ui}"
  Retrieved Information:"{'- ' + '\\n- '.join(retrieved_answers)}"
  Please synthesize this information and provide a clear answer to the user's query, strictly using the provided context.
  """
  result = gemini_model.generate_content(prompt_ui)
  return result.text

gr.Interface(fn=search_ui, inputs="text", outputs="text").launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://46da63ac83917f4ead.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Query: I feel angry all the time. i need dr for this

Best matching questions and their answers:


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1671.61ms
